# Task 1

During this first task we will apply some **pre-processing** operations to our raw data. So we will directly modify the data in *data/dataset/*, so that the data in *data/dataset_fixed/* will always remain unmodified.

## Reading the Datasets

At first, let's read the **.csv** files using a *pandas* function called **read_csv** whose parameters are:
- *filepath*: the path of the .csv files.
- *header*: will be always 0 to skip the columns' name row.
- *names*: gives a name to each column of the table.


In [1]:
import pandas as pd
import numpy as np

data_path = "../../data/dataset/sample"

ap = pd.read_csv(data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso'])
diag = pd.read_csv(data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei = pd.read_csv(data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf = pd.read_csv(data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd = pd.read_csv(data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [9]:
# Display example observations
print(ap.shape)
ap.head()

(250000, 12)


,idcentro,idana,sesso,annodiagnosidiabete,tipodiabete,scolarita,statocivile,professione,origine,annonascita,annoprimoaccesso,annodecesso
0,234,5697,M,2009.0,5,NaN,NaN,NaN,NaN,1949,2011.0,NaN
1,66,3539,M,2011.0,5,NaN,NaN,NaN,NaN,1933,2012.0,2015.0
2,100,3080,M,1988.0,5,NaN,NaN,NaN,NaN,1936,1991.0,NaN
3,112,3261,F,2014.0,5,NaN,NaN,NaN,NaN,1945,2014.0,NaN
4,160,9460,F,2014.0,5,NaN,NaN,NaN,NaN,1970,2016.0,NaN


In [3]:
print(diag.shape)
diag.head()

(4978504, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,1980-01-01,AMD247,491.20
1,1,5,1986-01-01,AMD247,272.0
2,1,5,1987-01-01,AMD083,NaN
3,1,5,1987-01-01,AMD247,401
4,1,5,1997-12-01,AMD247,410


In [4]:
print(elp.shape)
elp.head()

(29398377, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD001,169.0
1,1,5,2005-01-18,AMD002,76.0
2,1,5,2005-01-18,AMD004,135.0
3,1,5,2005-01-18,AMD005,80.0
4,1,5,2005-06-06,AMD007,169.0


In [5]:
print(ei.shape)
ei.head()

(1062856, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2006-01-04,AMD051,N
1,1,5,2006-11-14,AMD041,P
2,1,5,2006-11-20,AMD040,P
3,1,5,2007-06-01,AMD040,N
4,1,5,2008-01-16,AMD040,N


In [6]:
print(pdf.shape)
pdf.head()

(7289926, 7)


,idcentro,idana,data,codiceatc,quantita,idpasto,descrizionefarmaco
0,82,782,2008-04-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG
1,82,782,2008-04-02,A10BA02,1.0,3,GLUCOPHAGE*60CPR RIV 1000MG
2,82,782,2008-04-02,A10BA02,1.0,5,GLUCOPHAGE*60CPR RIV 1000MG
3,82,782,2008-04-02,A10BB09,3.0,1,DIAMICRON*60CPR R.M. 30MG
4,82,782,2008-05-02,A10BA02,1.0,1,GLUCOPHAGE*60CPR RIV 1000MG


In [7]:
print(pdnf.shape)
pdnf.head()

(569353, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2008-06-20,AMD152,NaN
1,1,5,2013-08-27,AMD152,NaN
2,1,5,2013-12-31,AMD086,S
3,1,5,2013-12-31,AMD228,S
4,1,34,2005-01-07,AMD090,S


In [10]:
print(pnd.shape)
pnd.head()

(5573033, 5)


,idcentro,idana,data,codiceamd,valore
0,1,5,2005-01-18,AMD121,C09AA05
1,1,5,2005-01-18,AMD124,C10AA05
2,1,5,2005-01-18,AMD124,C10AX06
3,1,5,2005-06-21,AMD121,C09AA05
4,1,5,2005-06-21,AMD124,C10AA05


## Exercise 1

**Select Events of Interest -**
We want only patients with at leats one cardiovascular event in their trajectories. These events have the folloging codes:
- **AMD047**: Myocardial infarction
- **AMD048**: Coronary angioplasty
- **AMD049**: Coronary bypass
- **AMD071**: Ictus
- **AMD081**: Lower limb angioplasty
- **AMD082**: Peripheral By-pass Lower Limbs
- **AMD208**: Revascularization of intracranial and neck vessels
- **AMD303**: Ischemic stroke

Now let's try to filter all the tables containing the parameter **codiceamd** so as to select just the rows describing one cardio-vascular event.


In [11]:
cardVasc_codes = ['AMD047', 'AMD048', 'AMD049', 'AMD071', 'AMD081', 'AMD082', 'AMD208', 'AMD303']

## pnd_noEvents = pnd[pnd.codiceamd.isin(cardVasc_codes)]
## pnd_noEvents.head()

## pdnf_noEvents = pdnf[pdnf.codiceamd.isin(cardVasc_codes)]
## pdnf_noEvents.head()

## ei_noEvents = ei[ei.codiceamd.isin(cardVasc_codes)]
## ei_noEvents.head()

## elp_noEvents = elp[elp.codiceamd.isin(cardVasc_codes)]
## elp_noEvents.head()

diag_cardEvents = diag[diag.codiceamd.isin(cardVasc_codes)]
print("BEFORE pre-processing: ", diag.shape)
print("AFTER  pre-processing: ", diag_cardEvents.shape)
diag_cardEvents.head()


BEFORE pre-processing:  (4978504, 5)
AFTER  pre-processing:  (328478, 5)


,idcentro,idana,data,codiceamd,valore
7,1,5,2000-12-02,AMD049,S
9,1,5,2001-03-22,AMD049,S
13,1,5,2002-02-11,AMD049,S
15,1,5,2002-02-14,AMD049,S
17,1,5,2003-01-17,AMD049,S


We realise that the only table with these cardio-vascular events is **diagnosi**, so we create a new DataFrame with all the rows that has one of these events, and we call it: **diag_noEvents**. And from this table, we extract the id's of the patients that have suffered at least one cardio-vascular event during his life:

In [12]:
patientsOfInterest = diag_cardEvents.filter(['idcentro','idana']).drop_duplicates()
print("Number of patients with at least one cardio-vascular event: ", patientsOfInterest.shape[0], "/", ap.shape[0] , "patients.")

Number of patients with at least one cardio-vascular event:  70193 / 250000 patients.


Finally, we have to delete from the rest of the tables all the information which is not associated to the remaining patients:

In [21]:
ap_new = pd.merge(ap, patientsOfInterest, on = ['idcentro','idana'])
diag_new = pd.merge(diag, patientsOfInterest, on = ['idcentro','idana'])
elp_new = pd.merge(elp, patientsOfInterest, on = ['idcentro','idana'])
ei_new = pd.merge(ei, patientsOfInterest, on = ['idcentro','idana'])
pdf_new = pd.merge(pdf, patientsOfInterest, on = ['idcentro','idana'])
pdnf_new = pd.merge(pdnf, patientsOfInterest, on = ['idcentro','idana'])
pnd_new =pd.merge(pnd, patientsOfInterest, on = ['idcentro','idana'])


## Exercise 4

**Modify the actual ranges of** ***esamilaboratorioparameteri*** **-** see this Table:

| Code | Descriptive Name | True Range |
|----------|----------|----------|
| AMD004 | Systolic blood pressure | 40 <= x <= 200 |
| AMD005 | Diastolic blood pressure | 40 <= x <= 130 |
| AMD007 | Fasting blood glucose | 50 <= x <= 500 |
| AMD008 | HbAlc | 5 <= x <= 15 |
| AMD009 | Creatininemia | Not Available |
| AMD111 | Microalbuminuria | Not Available |
| STITCH001 | BMI | Not Available |
| STITCH002 | LDL Choresterlo | 30 <= x <= 30|
| STITCH003 | Non-HDL Cholesterlo | 60 <= x <= 330 |
| STITCH004 | eGFR MDRD | Not Available |
| STITCH005 | eGFR CKD-EPI | Not Available |

In [27]:
NAN = np.nan
newRanges = [['AMD004', 'AMD005', 'AMD007', 'AMD008', 'AMD009', 'AMD111', 'STITCH001', 'STITCH002', 'STITCH003', 'STITCH004', 'STITCH005'],
          [40.0,40,50.0,5.0,NAN,NAN,NAN,30.0,60.0,NAN,NAN],
          [200.0,130.0,500.0,15.0,NAN,NAN,NAN,300.0,330.0,NAN,NAN]]

indexes = []

for k in range(len(newRanges[0])):
    if newRanges[1][k] != NAN:
        indexes += elp_new[(elp_new['codiceamd']==newRanges[0][k]) & ((elp_new['valore']<newRanges[1][k]) | (elp_new['valore']>newRanges[2][k]))].index.tolist()

elp_new_new = elp_new.drop(labels=indexes)

print("So now we have ", elp_new_new.shape[0], "/", elp_new.shape[0], " remaining occurrences (",elp_new.shape[0]-elp_new_new.shape[0],"deleted ).")


So now we have  10358221 / 10372962  remaining occurrences ( 14741 deleted ).
